In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from bs4 import BeautifulSoup, Tag
import re

In [ ]:
!pip install pythonbible

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 3.8 MB/s eta 0:00:00


In [ ]:
import plotly.graph_objs as go
import plotly.express as px
import pythonbible as bible

Citation Viz Data Collection

In [ ]:
!unzip /content/tlg0018_first1K-20231003T211502Z-001.zip

In [ ]:
import glob
grc_files = sorted(glob.glob('/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/*opp-grc1.xml'))
grc_files

['/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg001.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg002.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg003.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg004.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg005.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg006.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg007.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg008.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg009.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg010.opp-grc1.xml',
 '/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg011.opp

In [ ]:
def open_file(x):
  philo = open( x ,encoding='utf-8', errors='ignore')
  philo_soup = BeautifulSoup(philo,'lxml')
  return philo_soup


In [ ]:
def dir_name(x):
  philo = open(x, encoding='utf-8', errors='ignore')
  nom= philo.name[60:] #arbitrary length..the names of my xml files are very long
  return nom

In [ ]:
def make_refdict(soup):
  ref_dict={}
  for d in opened.find_all('div'):
    ref_dict[d['n']]= d.get_text()
  return ref_dict

In [ ]:
def pull_citations(soup_dict):
  cits= []
  pattern= '\s*\((.*?)\)\s*'  #pattern generated by chatGPT
  for v in soup_dict.values():
    match= re.findall(pattern,v)
    if match:
      cits.append(match)
  return cits
#list of lists

In [ ]:
def list_cleaning(lst_1):
  split_lst= [y.split("','") for x in lst_1 for y in x]
  flat_list = [item for sublist in split_lst for item in sublist]
  return flat_list #still a list of lists

In [ ]:
def ibid_corrections(lst_4): #code by ChatGPT
  prev_citation= None
  for i in range(len(lst_4)):
        if 'ib' in lst_4[i]:
            if prev_citation is not None:
                lst_4[i] = prev_citation
        else:
            prev_citation = lst_4[i]
  return lst_4

In [ ]:
def format_citations(lst3):
  citations= []
  try:
    for v in lst3:
      references= bible.get_references(v)
      formatted_reference= bible.format_scripture_references(references)
      citations.append(formatted_reference)
  except:
    pass
  return citations

In [ ]:
def splitting_chapter_verses(lst):
  splits= []
  split_2= []
  for item in lst:
    split= item.split(':')
    splits.append(split)
  return splits
  for item in splits:
    for y in item:
      again= y.split(';')
      split_2.append(again)
  return split_2





In [ ]:
def flatten_chap_vers(lst):
  flat= [item for sublist in lst for item in sublist]
  return flat

In [ ]:
def remove_verses(lst):
  for item in lst:
    if '-' in item:
      lst.remove(item)
  return lst

In [ ]:
def make_df(lst_6,p):
  couns= pd.Series(lst_6).value_counts()
  counts= pd.DataFrame(couns)
  #counts.reset_index(inplace=True)
  #counts.rename(columns={'index': "Citation", 0: "count"})
  counts.to_csv(f'/content/drive/MyDrive/Thesis2023/CSV_Files/Citations/{p}.csv')


In [ ]:
for x in grc_files:
  opened= open_file(x)
  named= dir_name(x)
  ref_dict= make_refdict(opened)
  cited= pull_citations(ref_dict)
  cleaned= list_cleaning(cited)
  ibids= ibid_corrections(cleaned)
  formatted= format_citations(ibids)
  chaps= splitting_chapter_verses(formatted)
  flattened= flatten_chap_vers(chaps)
  jchaps= remove_verses(flattened)
  frame= make_df(jchaps,named)

In [ ]:
opened= open_file('/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg002.opp-grc1.xml')
named= dir_name('/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg002.opp-grc1.xml')
ref_dict= make_refdict(opened)
cited= pull_citations(ref_dict)
cleaned= list_cleaning(cited)
ibids= ibid_corrections(cleaned)
formatted= format_citations(ibids)
chaps= splitting_chapter_verses(formatted)
flattened= flatten_chap_vers(chaps)
jchaps= remove_verses(flattened)
#frame= make_df(jchaps,named)

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [ ]:
jchaps

['Genesis 2',
 'Genesis 2',
 '',
 '',
 '',
 '',
 '',
 'Proverbs',
 '',
 'Genesis 2',
 '',
 'Genesis 2',
 'Numbers 6',
 'Numbers 6',
 'Genesis 2',
 'Genesis 2',
 '',
 '',
 'Genesis 2',
 '5',
 '',
 '',
 '',
 '',
 '',
 '',
 'Genesis 2',
 'Genesis 2',
 'Genesis 1',
 '',
 '',
 '',
 'Exodus 7',
 'Genesis 2',
 'Genesis 2',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Exodus 20',
 '',
 'Leviticus 19',
 'Genesis 2',
 '',
 '',
 'Genesis 2',
 '',
 '',
 'Genesis 42',
 'Genesis 2',
 '',
 '',
 '',
 '',
 '',
 'Numbers 12',
 'Genesis 2',
 'Genesis 2',
 'Genesis 2',
 'Genesis 29',
 'Genesis 49',
 'Genesis 30',
 'Exodus 28',
 '',
 '',
 'Genesis 2',
 'Genesis 2',
 '',
 '',
 '',
 '',
 '',
 'Genesis 3',
 'Genesis 2',
 '',
 '',
 'Genesis 2',
 '',
 '',
 'Genesis 2',
 '',
 '',
 '',
 '',
 'Genesis 2',
 '',
 '',
 '',
 '',
 'Genesis 2',
 '',
 '',
 '',
 '',
 '',
 'Genesis 1',
 '',
 'Genesis 2',
 '',
 'Deuteronomy 23',
 'Genesis 2',
 '',
 '',
 'Exodus 12',
 'Genesis 2',
 'Numbers 31',
 'Genesis 2',
 '',
 ''

In [ ]:
chapters= pd.Series(jchaps).value_counts()
chapters
chapters_df= pd.DataFrame(chapters)
chapters_df

,0
,188
Genesis 3,47
Genesis 2,35
Exodus 16,14
Genesis 15,10
...,...
23,1
Numbers 31,1
Exodus 7,1
"1,25",1


In [ ]:
chapters_df.reset_index(inplace=True)

In [ ]:
chapters_df

,index,0
0,,188
1,Genesis 3,47
2,Genesis 2,35
3,Exodus 16,14
4,Genesis 15,10
...,...,...
93,23,1
94,Numbers 31,1
95,Exodus 7,1
96,"1,25",1


In [ ]:
new= chapters_df.rename(columns={'index':'Citation',0: "Count"})

In [ ]:
new

,Citation,Count
0,,188
1,Genesis 3,47
2,Genesis 2,35
3,Exodus 16,14
4,Genesis 15,10
...,...,...
93,23,1
94,Numbers 31,1
95,Exodus 7,1
96,"1,25",1


In [ ]:
chapters_df.name= 'Allegorical Interpretations, Books 1-3'

In [ ]:
new1= new.dropna()

In [ ]:
new1= new.drop(0)

In [ ]:
new1

,Citation,Count
1,Genesis 3,47
2,Genesis 2,35
3,Exodus 16,14
4,Genesis 15,10
5,Numbers 12,10
...,...,...
93,23,1
94,Numbers 31,1
95,Exodus 7,1
96,"1,25",1


In [ ]:
fig = px.treemap(new1, path=[px.Constant(f'{chapters_df.name}'), 'Citation'],
                 values='Count', color='Citation')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
opened= open_file('/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg008.opp-grc1.xml')
named= dir_name('/content/drive/MyDrive/Thesis2023/XML Files/tlg0018_first1K/tlg0018.tlg008.opp-grc1.xml')
ref_dict= make_refdict(opened)
cited= pull_citations(ref_dict)
cleaned= list_cleaning(cited)
ibids= ibid_corrections(cleaned)
formatted= format_citations(ibids)
chaps= splitting_chapter_verses(formatted)
flattened= flatten_chap_vers(chaps)
jchaps= remove_verses(flattened)
#frame= make_df(jchaps,named)

/usr/local/lib/python3.10/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning:

It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.



In [ ]:
_8= pd.Series(jchaps).value_counts()
df_8= pd.DataFrame(_8)
df_8

,0
,20
Genesis 6,4
Genesis 2,1
"9,22",1
Genesis 15,1
Numbers 28,1
Genesis 38,1
Deuteronomy 5,1
Deuteronomy 15,1
"19,30",1


In [ ]:
df_8_1= df_8.rename(columns={'index':'Citation',0: "Count"})
df_8_1

,Count
,20
Genesis 6,4
Genesis 2,1
"9,22",1
Genesis 15,1
Numbers 28,1
Genesis 38,1
Deuteronomy 5,1
Deuteronomy 15,1
"19,30",1
